In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

data = pd.concat([train, test], sort=False)

data

In [ ]:
#Plot features with more than 1000 null values

features =[]
nullValues =[]
for i in data:
    if (data.isna().sum()[i])>1000 and i!='SalePrice':
        features.append(i)
        nullValues.append(data.isna().sum()[i])
y_pos = np.arange(len(features))
plt.bar(y_pos, nullValues, align='center' , alpha=0.5)
plt.xticks(y_pos, features)
plt.ylabel('NULL values')
plt.xlabel('Features')
plt.title('Features with more than 1000 NULL values')
plt.show()

In [ ]:

# Drop columns that contain more than 1000 NULL values
data = data.dropna(axis=1, thresh=1000)

# Fill only the numeric columns' NULL values with their mean
numeric_cols = data.select_dtypes(include=[np.number]).columns
data.loc[:, numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

data



In [5]:
#Dealing with NULL values
data = pd.get_dummies(data) #Convert string values to integer values

In [6]:
#Drop features that are correlated to each other
covarianceMatrix = data.corr()
listOfFeatures = [i for i in covarianceMatrix]
setOfDroppedFeatures = set()
for i in range(len(listOfFeatures)) :
    for j in range(i+1, len(listOfFeatures)):
        feature1=listOfFeatures [i]
        feature2=listOfFeatures [j]
        if abs (covarianceMatrix[feature1 ] [ feature2]) > 0.8: #If the correlation between the features is > 0.8
            setOfDroppedFeatures. add (feature1)
#I tried different values of threshold and 0.8 was the one that gave the best results
data = data.drop(setOfDroppedFeatures, axis=1)


In [7]:
#Drop features that are not correlated with output
nonCorrelatedWithOutput = [column for column in data if abs(data[column] . corr(data["SalePrice"]) ) < 0.045]
#I tried different values of threshold and 0.045 was the one that gave the best results
data = data.drop(nonCorrelatedWithOutput, axis=1)

In [ ]:
#Plot one of the features with outliers
plt.plot(data['LotArea'], data['SalePrice'], 'bo')
plt.axvline(x=75000, color='r')
plt.ylabel('SalePrice')
plt.xlabel('LotArea')
plt. title('SalePrice in function of LotArea')
plt.show()

In [ ]:

newTrain = data.iloc[:1460]
newTest = data.iloc[1460:]


def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])  
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)  
    upper_bound = quartile_3 + (iqr * 1.5)  
    return np.where((ys > upper_bound) | (ys < lower_bound)) 


trainWithoutOutliers = newTrain.copy()  
numeric_columns = newTrain.select_dtypes(include=[np.number]).columns  

for column in numeric_columns:
    outlierIndices = outliers_iqr(newTrain[column])[0]  
    outlierValuesList = np.ndarray.tolist(outlierIndices)  
    
    
    trainWithoutOutliers = trainWithoutOutliers.drop(outlierValuesList, axis=0, errors='ignore')


trainWithoutOutliers



In [10]:
X = trainWithoutOutliers.drop("SalePrice",axis=1) 
Y = np.log1p(trainWithoutOutliers["SalePrice"]) #Get SalePrice column {log1p(x) = log(x+1)}
reg = LinearRegression().fit(X, Y)

In [ ]:

# Check if "SalePrice" exists before dropping


if "SalePrice" in newTest.columns:
    newTest = newTest.drop("SalePrice", axis=1) 

pred = np.expm1(reg.predict(newTest))


sub = pd.DataFrame()  
sub['Id'] = test['Id']  
sub['SalePrice'] = pred  
sub.to_csv("submission.csv", index=False)  

sub